# Glove 실습

In [6]:
import numpy as np
import os 
import pickle
import time

SENTENCE_LENGTH_MAX = 32
EMBEDDING_DIM=50

import nltk
nltk.download('punkt')
sentence_splitter = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82102\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def corpus_sentence_tokens(corpus_text_file):
    while True:
        with open(corpus_text_file,encoding = 'utf8') as f:
            for line in f.readlines():
                n,l = line.split('\t')# Strip of the initial numbers
                for s in sentence_splitter.tokenize(l): # Split the lines into sentences (~1 each)
                    tree_banked = tokenizer.tokenize(s)
                if len(tree_banked) < SENTENCE_LENGTH_MAX:
                    yield tree_banked

corpus_sentence_tokens_gen = corpus_sentence_tokens('en.wikipedia.2010.100K.txt')

In [4]:
import glove
glove_corpus = glove.Corpus()
corpus_sentences = [
        [ w.lower() for w in next(corpus_sentence_tokens_gen)] # All lower-case
        for _ in range(0,100*1000)
    ]
# Fit the co-occurrence matrix using a sliding window of 10 words.
t0 = time.time()
glove_corpus.fit(corpus_sentences, window=10)

print("Dictionary length=%d" % (len(glove_corpus.dictionary),))
print("Co-occurrence calculated in %5.1fsec" % (time.time()-t0, ))

C:\Users\82102\Anaconda3\envs\my_python\lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\82102\Anaconda3\envs\my_python\lib\site-packages\glove\glove_inner.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


AttributeError: module 'glove' has no attribute 'Corpus'

In [5]:
word_embedding = glove.Glove(no_components=EMBEDDING_DIM, learning_rate=0.05)
t0 = time.time()
glove_epochs=20
glove_threads=4
word_embedding.fit(glove_corpus.matrix, epochs=glove_epochs, no_threads=glove_threads, verbose=True)
print("%d-d word-embedding created in %5.1fsec = %5.1fsec per epoch" % (
        EMBEDDING_DIM, (time.time()-t0), (time.time()-t0)/glove_epochs*glove_threads, ))
# Add the word -> id dictionary to the model to allow similarity queries.
word_embedding.add_dictionary(glove_corpus.dictionary)

NameError: name 'EMBEDDING_DIM' is not defined

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']

# Fast_Text

In [2]:
# 구글 드라이브에서 csv 파일을 읽어오기 위해 gauth 인증을 합니다.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
id = '1oL8oLH-uw76zFzMFjJd75hraR8zriJeR'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('text8')

ModuleNotFoundError: No module named 'google.colab'

In [5]:
from gensim.models.word2vec import Text8Corpus
from gensim.models import FastText

ft_model = FastText(Text8Corpus('1_text8'),size =100,window = 5 ,min_count=150,workers=4, min_n=3, max_n=10)

In [6]:
print(ft_model.most_similar(['woman','king'],['man'],topn=3))

C:\Users\82102\Anaconda3\envs\my_python\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('emperor', 0.695671796798706), ('prince', 0.6808103919029236), ('elizabeth', 0.6729668378829956)]
